In [ ]:
# Define the model using functional API:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Input,Reshape,Flatten
import keras
import numpy as np

layers = [Input(shape=(256,256,1)),
      Reshape((64,64,-1)),
      Conv2D(filters=32,kernel_size=5),
      MaxPool2D(2),
      Conv2D(filters=1,kernel_size=1),
      Flatten(),
      Dense(16,activation='sigmoid')]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 64, 64, 16)        0         
                                                                 
 conv2d (Conv2D)             (None, 60, 60, 32)        12832     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 1)         33        
                                                                 
 flatten (Flatten)           (None, 900)               0         
                                                                 
 dense (Dense)               (None, 16)                14416     
                                                        

In [ ]:
# Define the model using functional API:
from keras.models import Model
from keras.layers import Dense,Conv2D,MaxPool2D,Input,Reshape,Flatten,concatenate
import keras
import numpy as np

def my_model():
  # define layers
  input_tensor = Input(shape=(256,256,1))
  reshape_layer = Reshape((64,64,-1))
  conv2d_1 = Conv2D(filters=8,kernel_size=5)
  conv2d_2 = Conv2D(filters=8,kernel_size=17)
  max_pool_layer = MaxPool2D(4)

  input_tensor_reshape = reshape_layer(input_tensor)
  conv1_out = conv2d_1(input_tensor_reshape)
  conv2_out = conv2d_2(input_tensor)
  max_pool_out = max_pool_layer(conv2_out)
  cat_out = concatenate([conv1_out,max_pool_out],axis=-1)
  model = Model(inputs=input_tensor, outputs=cat_out)
  return model

my_model = my_model()
my_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 reshape_2 (Reshape)            (None, 64, 64, 16)   0           ['input_6[0][0]']                
                                                                                                  
 conv2d_5 (Conv2D)              (None, 240, 240, 8)  2320        ['input_6[0][0]']                
                                                                                                  
 conv2d_4 (Conv2D)              (None, 60, 60, 8)    3208        ['reshape_2[0][0]']          

In [ ]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Input
from keras.losses import SparseCategoricalCrossentropy

df_orig_train = pd.read_csv('/content/drive/MyDrive/keras/mnist_train.csv',header=None)
df_orig_test = pd.read_csv('/content/drive/MyDrive/keras/mnist_test.csv',header=None)
df_train_values = df_orig_train.values
df_test_values = df_orig_test.values
train_feat_ori,train_label_ori = df_train_values[:,1:]/255.0,df_train_values[:,0]
test_feat,test_label = df_test_values[:,1:]/255.0,df_test_values[:,0]
train_feat,val_feat = train_feat_ori[6000:],train_feat_ori[:6000]
train_label,val_label = train_label_ori[6000:],train_label_ori[:6000]

model = Sequential()
model.add(Input(shape=(784,)))
model.add(Dense(512, activation='relu')) 
model.add(Dense(512, activation='relu')) 
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',loss= 'sparse_categorical_crossentropy',metrics=['accuracy'])

batch_size = 128
epochs = 8
model.fit(train_feat, train_label, batch_size=batch_size, epochs=epochs, validation_data = (val_feat,val_label))

Epoch 1/8
422/422 [==============================] - 11s 24ms/step - loss: 0.2306 - accuracy: 0.9338 - val_loss: 0.1103 - val_accuracy: 0.9660
Epoch 2/8
422/422 [==============================] - 8s 18ms/step - loss: 0.0842 - accuracy: 0.9738 - val_loss: 0.0790 - val_accuracy: 0.9742
Epoch 3/8
422/422 [==============================] - 8s 18ms/step - loss: 0.0528 - accuracy: 0.9833 - val_loss: 0.0613 - val_accuracy: 0.9800
Epoch 4/8
422/422 [==============================] - 8s 18ms/step - loss: 0.0363 - accuracy: 0.9884 - val_loss: 0.0762 - val_accuracy: 0.9790
Epoch 5/8
422/422 [==============================] - 8s 18ms/step - loss: 0.0285 - accuracy: 0.9911 - val_loss: 0.0799 - val_accuracy: 0.9783
Epoch 6/8
422/422 [==============================] - 8s 18ms/step - loss: 0.0213 - accuracy: 0.9929 - val_loss: 0.0757 - val_accuracy: 0.9797
Epoch 7/8
422/422 [==============================] - 8s 18ms/step - loss: 0.0177 - accuracy: 0.9944 - val_loss: 0.0770 - val_accuracy: 0.9815
Epoch

In [3]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Input
from keras.losses import SparseCategoricalCrossentropy

import numpy as np
import linecache
import random
class DataGenerator(keras.utils.Sequence):
  'Generates data for Keras'
  def __init__(self, csv_path, indexes, bs):
    # initilizes some variables 
    self.csv_path = csv_path
    self.norm_facor = 255.0
    self.indexes = indexes
    self.bs = bs
    random.shuffle(self.indexes)
  def __len__(self):
    # return the total number of samples in the dataset
    return (len(self.indexes))//self.bs-1
  def __getitem__(self,index):
    # get one sample according to the index
    feat_all = []
    label_all = []
    for this_index in range(index*self.bs,(index+1)*self.bs):
      line_index = self.indexes[index]
      line_str = linecache.getline(self.csv_path,line_index)
      line_val = [int(i) for i in line_str.split(',') if len(i)]
      label = line_val[0]
      feat = np.array(line_val[1:])/self.norm_facor
      feat_all.append(feat)
      label_all.append(label)

    return np.array(feat_all),np.array(label_all)
  def shuffle(self):
    random.shuffle(self.indexes)


indexes = [i for i in range(60000)]
train_index = indexes[6000:]
val_index = indexes[:6000]
batch_size = 128
train_set = DataGenerator('/content/drive/MyDrive/keras/mnist_train.csv',train_index,batch_size)
val_set = DataGenerator('/content/drive/MyDrive/keras/mnist_train.csv',val_index,batch_size)

model = Sequential()
model.add(Input(shape=(784,)))
model.add(Dense(512, activation='relu')) 
model.add(Dense(512, activation='relu')) 
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',loss= 'sparse_categorical_crossentropy',metrics=['accuracy'])

epochs = 15
for i_epoch in range(epochs):
  print('[{}/{}]'.format(i_epoch,epochs))
  model.fit(train_set, steps_per_epoch = len(train_set), epochs=1, validation_data = val_set,validation_steps=len(val_set))
  train_set.shuffle()

[0/15]
420/420 [==============================] - 28s 65ms/step - loss: 1.2024 - accuracy: 0.6071 - val_loss: 0.3391 - val_accuracy: 0.8444
[1/15]
420/420 [==============================] - 27s 65ms/step - loss: 0.7259 - accuracy: 0.7738 - val_loss: 0.3641 - val_accuracy: 0.9111
[2/15]
420/420 [==============================] - 27s 65ms/step - loss: 0.5904 - accuracy: 0.8238 - val_loss: 0.3575 - val_accuracy: 0.8444
[3/15]
420/420 [==============================] - 27s 65ms/step - loss: 0.5424 - accuracy: 0.8452 - val_loss: 0.1123 - val_accuracy: 1.0000
[4/15]
420/420 [==============================] - 30s 71ms/step - loss: 0.5796 - accuracy: 0.8429 - val_loss: 0.1937 - val_accuracy: 0.9556
[5/15]
420/420 [==============================] - 27s 64ms/step - loss: 0.4774 - accuracy: 0.8643 - val_loss: 0.1610 - val_accuracy: 0.9333
[6/15]
420/420 [==============================] - 30s 71ms/step - loss: 0.3871 - accuracy: 0.8881 - val_loss: 0.2727 - val_accuracy: 0.9333
[7/15]
420/420 [====

In [ ]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Input

import numpy as np
import linecache
import random
import tensorflow as tf

class DataGenerator(keras.utils.Sequence):
  'Generates data for Keras'
  def __init__(self, csv_path, indexes, bs):
    # initilizes some variables 
    self.csv_path = csv_path
    self.norm_facor = 255.0
    self.indexes = indexes
    self.bs = bs
    random.shuffle(self.indexes)
  def __len__(self):
    # return the total number of samples in the dataset
    return (len(self.indexes))//self.bs-1
  def __getitem__(self,index):
    # get one sample according to the index
    feat_all = []
    label_all = []
    for this_index in range(index*self.bs,(index+1)*self.bs):
      line_index = self.indexes[index]
      line_str = linecache.getline(self.csv_path,line_index)
      line_val = [int(i) for i in line_str.split(',') if len(i)]
      label = line_val[0]
      feat = np.array(line_val[1:])/self.norm_facor
      feat_all.append(feat)
      label_all.append(label)

    return np.array(feat_all),np.array(label_all)
  def shuffle(self):
    random.shuffle(self.indexes)


indexes = [i for i in range(60000)]
train_index = indexes[6000:]
val_index = indexes[:6000]
batch_size = 128
train_set = DataGenerator('/content/drive/MyDrive/keras/mnist_train.csv',train_index,batch_size)
val_set = DataGenerator('/content/drive/MyDrive/keras/mnist_train.csv',val_index,batch_size)

model = Sequential()
model.add(Input(shape=(784,)))
model.add(Dense(512, activation='relu')) 
model.add(Dense(512, activation='relu')) 
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',loss= 'sparse_categorical_crossentropy',metrics=['accuracy'])

loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Adam()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()
epochs = 50
best_metric = 100
for epoch in range(epochs):
  train_set.shuffle()
  for step, (x_batch_train, y_batch_train) in enumerate(train_set):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
  for x_batch_val, y_batch_val in val_set:
    val_logits = model(x_batch_val, training=False)
    val_acc_metric.update_state(y_batch_val, val_logits)
  val_acc = val_acc_metric.result()
  print("[{}/{}],val_acc:{}".format(epoch,epochs,float(val_acc)))
  if float(val_acc) < best_metric:
    best_metric = float(val_acc)
    model.save('path_to_location.h5')
  val_acc_metric.reset_states()



[0/50],val_acc:0.8888888955116272
[1/50],val_acc:0.9333333373069763


In [1]:
from keras.layers import Dense,LSTM
import keras
import numpy as np

x = keras.backend.constant(np.random.randn(1,100,128))
lstm_layer1 = LSTM(512,return_sequences=True)
lstm_layer2 = LSTM(512,return_sequences=False)
y1 = lstm_layer1(x)
y2 = lstm_layer2(x)
for ii in [x,y1,y2]:
  print(ii.shape)


(1, 100, 128)
(1, 100, 512)
(1, 512)


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Input,Conv2D, MaxPool2D,Flatten,Reshape

layer = [
    Input(shape=(256,256,1)),
    Reshape((64,64,-1)),
    Conv2D(32,5),
    MaxPool2D(2),
    Conv2D(1,1),
    Flatten(),
    Dense(16,activation ='sigmoid')
]
model = Sequential(layer)



In [4]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 64, 64, 16)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 60, 60, 32)        12832     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 30, 30, 1)         33        
                                                                 
 flatten_2 (Flatten)         (None, 900)               0         
                                                                 
 dense_2 (Dense)             (None, 16)                14416     
                                                      

In [21]:
from keras.layers import Dense, Input,Conv2D, MaxPool2D,Flatten,Reshape,concatenate
from keras.models import Model
def new_modle():
    input_tensor = Input(shape=(256,256,1))
    reshape_layer = Reshape((64,64,-1))
    conv1 = Conv2D(8,5)
    conv2 = Conv2D(8,17)
    mxp = MaxPool2D(4)

    #
    reshaped = reshape_layer(input_tensor)
    concl_out = conv1(reshaped)
    conv2_out = conv2(input_tensor)
    mxp_out = mxp(conv2_out)
    out = concatenate([concl_out,mxp_out],axis= -1)

    model = Model( inputs = input_tensor,outputs = out)
    model.summary()

In [22]:
new_modle()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 reshape_11 (Reshape)           (None, 64, 64, 16)   0           ['input_12[0][0]']               
                                                                                                  
 conv2d_23 (Conv2D)             (None, 240, 240, 8)  2320        ['input_12[0][0]']               
                                                                                                  
 conv2d_22 (Conv2D)             (None, 60, 60, 8)    3208        ['reshape_11[0][0]']         